In [1]:
# Import the needed Packages
# FMU Simulation
import MoBASimulator as mb
# Numpy
import numpy as np
# Bokeh for Plotting
import bokeh.plotting as bk
import bokeh.io as bi
#from bokeh.io import export_svgs
bi.output_notebook()
import Algorithms as alg

# Model Parameter
k = [[10,0],[1,5]]
t = [[10,1],[5,12]]
l = [[0,0],[0,0]]

# The needed Parameter
K = np.zeros((2,2))
T = np.zeros((2,2))
L = np.zeros((2,2))


# Get the parameter names
params = {}
for outputs in range(1,3):
    for inputs in range(1,3):
        params.update({"fmu.K["+str(outputs)+","+str(inputs)+"]": k[outputs-1][inputs-1]})
        params.update({"fmu.T["+str(outputs)+","+str(inputs)+"]": t[outputs-1][inputs-1]})
        params.update({"fmu.L["+str(outputs)+","+str(inputs)+"]": l[outputs-1][inputs-1]})

# Load a Model
sim = mb.Simulator()
sim.loadModel("./Experiments/TITO/MIMO_0Study_TITO_0FOTD.fmu")
# Set Parameter
sim.set(params)
#sim.showParameterDialog()
#sim.getParameterNames(), params.keys()

# First run, Input 1 -> Output 1 & 2
sim.set({"fmu.u_1":1,"fmu.u_2":0})
# Set timestep = 1e-2, endtime = 100
res=sim.simulate(0.01, 200)

# Get the signals
y = res["fmu.y_1"]
y2 = res["fmu.y_2"]
u = res["fmu.u_1"]
t = res["time"]


# Get TF from 1-> 1
K[0][0],T[0][0],L[0][0] = alg.Integral_Identification(y,u,t)
# Get TF from 1-> 2
K[1][0],T[1][0],L[1][0] = alg.Integral_Identification(y2,u,t)

# Second run, Input 2 -> Output 1 & 2
# Reload the Model to set everything to zero
sim.resetModelState()
sim.set({"fmu.u_1":0, "fmu.u_2":1})
# Set timestep = 1e-2, endtime = 100
res=sim.simulate(0.01, 200)

# Get the signals
y = res["fmu.y_1"]
y2 = res["fmu.y_2"]
u = res["fmu.u_2"]
t = res["time"]

# Get TF from 2-> 1
K[0][1],T[0][1],L[0][1] = alg.Integral_Identification(y,u,t)
# Get TF from 2-> 2
K[1][1],T[1][1],L[1][1] = alg.Integral_Identification(y2,u,t)

K,T,L

Loading BokehJS ...

(array([[ 10.00002789,   1.00004676],
        [  0.        ,   4.9999655 ]]), array([[  9.99986163,   5.00030322],
        [  0.        ,  11.98298173]]), array([[ 0.0019066 ,  0.00165399],
        [ 0.        ,  0.01467551]]))

In [10]:
# Make a decentralized Controller
KY,KR = alg.Control_Decentral(K,T,L)
#KY,KR = alg.Control_Astrom(K,T,L,0.1*np.eye(2,2))
#KY,KR = alg.Control_Decoupled(K,T,L,0.1*np.eye(2,2))
KY, KR

(array([[[ 3.41691246,  2.85937075,  0.        ],
         [ 0.        ,  0.        ,  0.        ]],
 
        [[ 0.        ,  0.        ,  0.        ],
         [ 8.22144069,  6.76026247,  0.        ]]]),
 array([[[ 0.        ,  2.85937075,  0.        ],
         [ 0.        ,  0.        ,  0.        ]],
 
        [[ 0.        ,  0.        ,  0.        ],
         [ 0.        ,  6.76026247,  0.        ]]]))

In [14]:
KY,KR = alg.Control_Decoupled(K,T,L,0.1*np.eye(2,2))
KY[0][1]

array([-0.82218022, -0.67605597,  0.        ])

In [13]:
KY,KR = alg.Control_Astrom(K,T,L,0.1*np.eye(2,2))
KY,KR

(array([[[ 3.52307768,  2.95860305,  0.        ],
         [ 0.        , -0.13521213,  0.        ]],
 
        [[-0.82218022, -0.67605597,  0.        ],
         [ 8.22144069,  6.76026247,  0.        ]]]),
 array([[[ 0.        ,  2.94508124,  0.        ],
         [ 0.        ,  0.        ,  0.        ]],
 
        [[ 0.        , -0.67605597,  0.        ],
         [ 0.        ,  6.76026247,  0.        ]]]))